In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reduce Tech Debt with Gemini 3 Pro

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/refactor_legacy_code.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fcode%2Frefactor_legacy_code.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/code/refactor_legacy_code.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/refactor_legacy_code.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery Studio logo"><br> Open in BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/refactor_legacy_code.ipynb">
      <img width="32px" src="https://storage.googleapis.com/github-repo/generative-ai/logos/GitHub_Invertocat_Dark.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<p>
<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/refactor_legacy_code.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/refactor_legacy_code.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/refactor_legacy_code.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/refactor_legacy_code.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/refactor_legacy_code.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>
</p>

| Authors |
| --- |
| [Dennis Kashkin](https://github.com/dkashkin) |
| [Holt Skinner](https://github.com/holtskinner) |

## Overview: Escaping "Legacy Land"

We all know the code. The ten-year-old library that nobody quite understands anymore. You don't want to be the one who breaks it, but you can't ignore it forever. The struggle is so relatable that it inspired [Forrest, Amanda, and Holt to sing about it at Next '23](https://goo.gle/legacy-land).

But while the fear is real, so is the need for modernization. Security, performance, and scalability demand that we refactor.

**About this Notebook**
We are going to look at a new approach to this old problem. utilizing **Gemini 3 Pro on Vertex AI**. We will look at how to leverage the model not just for faster coding, but for analyzing dependencies and executing safer, more reliable code modernization.

## Prerequisites

Before starting, ensure you have the necessary dependencies and environment set up.

*(Note: In a Jupyter environment, you may need to restart the kernel after installing new packages.)*

In [ ]:
%pip install -q --upgrade google-genai pyparseit

### Authenticate your notebook environment

If you are running this notebook in **Google Colab**, run the cell below to authenticate your account.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
import os

from google import genai

# fmt: off
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
LOCATION = "global" # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))
if not LOCATION:
    LOCATION = os.environ.get("GOOGLE_CLOUD_REGION")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Load model

In [ ]:
MODEL_ID = "gemini-3-pro-preview"  # @param {type:"string"}

### Import libraries and create helper functions

In [ ]:
import base64
import logging
import re

import markdown
from IPython.display import HTML, display
from google.genai import types
from pyparseit import parse_markdown_string

logging.disable(level=logging.WARNING)


def call_llm(prompt: str) -> str:
    """Simple wrapper for model inference."""
    print(f"Calling {MODEL_ID} with {len(prompt)} chars...")
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config=types.GenerateContentConfig(
            temperature=1.0,
            thinking_config=types.ThinkingConfig(thinking_budget=16384),
        ),
    )
    # Basic cleanup of markdown code blocks if present
    snippets = parse_markdown_string(response.text, language="python")
    return snippets[0].content


def generate_and_save(template_path: str, output_path: str, replacements: dict) -> str:
    """Helper to load a prompt template, inject variables, call LLM, and save the result."""
    with open(template_path) as f:
        prompt = f.read()

    for key, value in replacements.items():
        prompt = prompt.replace(f"{{{key}}}", value)

    print(f"Generating from {template_path}...")
    content = call_llm(prompt)

    with open(output_path, "w") as f:
        f.write(content)
    return content


def render_markdown_with_mermaid(markdown_string: str) -> None:
    """Renders a string containing markdown and mermaid diagrams inline."""
    # Placeholder and storage logic is identical to the cell magic
    mermaid_placeholder = "---MERMAID_PLACEHOLDER_{}---"
    mermaid_blocks = re.findall(r"```mermaid\n(.*?)\n```", markdown_string, re.DOTALL)

    temp_md = markdown_string
    for i, block in enumerate(mermaid_blocks):
        temp_md = temp_md.replace(
            f"```mermaid\n{block}\n```", mermaid_placeholder.format(i), 1
        )

    html_output = markdown.markdown(
        temp_md, extensions=["extra", "fenced_code", "tables"]
    )

    for i, block in enumerate(mermaid_blocks):
        encoded_mermaid = base64.b64encode(block.encode("utf-8")).decode("utf-8")
        mermaid_html = f"""
        <div class='mermaid' style='margin-top: 1em; margin-bottom: 1em;'><iframe
            src="https://mermaid.ink/svg/{encoded_mermaid}"
            width="100%" height="450" frameborder="0" scrolling="no"
            style="max-width: 100%; display: block; margin: auto;"
        ></iframe></div>"""
        html_output = html_output.replace(mermaid_placeholder.format(i), mermaid_html)

    display(HTML(html_output))

### Create necessary directories.

In [ ]:
import os

DIRS = [
    "original_code",
    "original_tests",
    "output/tests",
    "output/specs",
    "output/design_docs",
    "output/code",
    "prompts",
]
for d in DIRS:
    os.makedirs(d, exist_ok=True)

#### Load Prompts

Load the specialized prompts for each stage of the pipeline.

In [ ]:
%%writefile prompts/create_tests.md
<role>
You are a world-class software engineer specialized in writing comprehensive unit tests for specific Python modules.
</role>

<task>
Your goal is to write a robust set of "Passing Tests" for the Python module provided in the `source_code` tag below.
"Passing Tests" are unit tests that cover existing, valid code paths and are expected to pass.
Do not write any "Failing Tests" that are expected to result in errors based on the current version of source code. For example, if you see insufficient validation of input parameters or lack of exception handling, do not write any tests that target such known gaps.
</task>

<instructions>
You must follow these instructions:
- You must use Python's standard framework "unittest"
- You must write a complete Python module including all required import statements
- Wrap all unit tests into one test class inherited from unittest.TestCase
- The source code of the tested module must be imported using the following exact code:

```python
import importlib
import os
import sys

sys.path.insert(0, os.getcwd())
PACKAGE_PREFIX = os.environ.get('PACKAGE_PREFIX', '{code_dir_path}')
module = importlib.import_module(f'{PACKAGE_PREFIX}.{module_name}')
```

- Include the following code at the very end of the Python module with the unit test class:

```python
if __name__ == '__main__':
    unittest.main()
```

</instructions>

<response_format>
Format your response as a single Markdown code block with the full source code of the Python module containing all unit tests.
</response_format>

<best_practices>
# A Comprehensive Guide to Unit Testing Python Modules

Writing effective unit tests is a critical skill in software engineering. When testing a Python module with multiple classes and functions, the scope expands from testing individual functions in isolation to verifying the collaborative behavior of those methods and managing the state of class instances. A key metric for a thorough test suite is code coverage, which measures the percentage of your code executed by your tests. While aiming for 100% coverage is an excellent goal that builds confidence and reduces bugs, remember that it is a tool, not the ultimate goal. A meaningful test that verifies the class's public behavior is always more valuable than a test that simply executes a line of code.

## 1. Understand the Role and Public Interface of every class and function

Before writing a single test, you must thoroughly understand the classes and functions you intend to test. This involves looking at the bigger picture of how each class and its methods are intended to be used.

*   **Analyze the Responsibilities**: Read the code, docstrings, and any related specifications to understand the purpose, the public methods (the intended interface), and the interactions with other components.
*   **Focus on Public Behavior**: Prioritize testing the public methods of classes, as these define how other parts of your code will interact with it. While it's possible to test private methods in Python, it's often a sign that your class may have too many responsibilities and could be a candidate for refactoring. Testing through the public interface helps ensure your tests aren't tightly coupled to implementation details, making them less brittle to future code changes.
*   **Identify Method Interactions and State Changes**: Map out how the methods within each class interact with each other. Does one method's execution affect the state of the object, which in turn influences the outcome of another method? These interactions are crucial to test.
*   **Map Out All Code Paths**: For each public method, and any critical private methods that contain complex logic, you must identify and test all execution paths created by conditional logic.
    *   **Conditionals**: Every `if`, `elif`, and `else` statement creates a branch that needs to be tested.
    *   **Loops**: Test scenarios where a loop runs zero, one, or multiple times. Also, test any `break` or `continue` statements that alter the loop's flow.
    *   **Exception Handling**: Ensure your tests trigger each `try` block's success path and every corresponding `except` block's failure path.
    *   **Early Returns**: Functions with guard clauses or multiple return statements have exits that must all be tested.

## 2. Design Test Cases Systematically

A structured approach to test case design ensures that you cover all expected behaviors, boundaries, and failure scenarios.

*   **Equivalence Partitioning**: Divide possible inputs for each method into groups that should be handled similarly. For example, a method processing a user's age might have partitions for valid ages (e.g., 18-65), underage (e.g., 0-17), and invalid inputs (e.g., negative numbers or non-integer types). Test one representative from each group.
*   **Boundary Value Analysis**: Pay special attention to the edges of your input ranges. For an age range of 18-65, you should test the values 17, 18, 65, and 66.
*   **Test the "Happy Path"**: Start with at least one test for the normal, expected use case of each public method with valid inputs. For a class, this may involve a sequence of method calls that represent a typical interaction.
*   **Cover Edge Cases and Invalid Inputs**: This is where many bugs hide. Your tests should cover:
    *   **Empty or Null Inputs**: `None`, empty strings (`""`), and empty collections (`[]`, `{}`).
    *   **Numeric Boundaries**: `0`, `-1`, and very large or small numbers.
    *   **Invalid Types**: Pass a string where an integer is expected to ensure proper error handling.
*   **Test State Transitions**: For stateful classes, design tests that specifically check the transition of the object's state after a method is called. For instance, if a `user.deactivate()` method is called, a subsequent call to `user.is_active()` should return `False`.

## 3. Leverage Mocking

Classes and functions often depend on external systems like databases, APIs, or the file system. To test your logic in isolation, you must use **mocking**. Mocking involves replacing these external dependencies with controlled objects that simulate their behavior.

*   **Why Mock?**: Mocking makes your tests faster, more reliable, and independent of external systems. It allows you to simulate success, failure (e.g., an API timeout), and edge case responses from dependencies without making real network or disk operations.
*   **How to Mock**: Python's `unittest.mock` library provides tools like `@patch` to replace objects within a test's scope. When mocking, it's a good practice to use `autospec=True` to ensure your mock has the same interface as the object it's replacing.
*   **Dependency Injection**: A powerful technique to make your functions and classes more testable is dependency injection. Instead of a class creating its own dependencies internally, you provide them as arguments to the constructor. This makes it easy to substitute a real dependency with a mock during testing.

**Example: Mocking an API Call in a Class Method**

```python
from unittest.mock import patch, Mock

# Assuming 'user_service.get_user_data' calls 'requests.get'
@patch('my_app.user_service.requests.get')
def test_get_user_data_success(mock_get):
    # Configure the mock to return a successful response
    mock_response = Mock()
    mock_response.json.return_value = {'id': 1, 'name': 'Jane Doe'}
    mock_get.return_value = mock_response

    # Instantiate the service and call the method
    service = UserService()
    user_data = service.get_user_data(1)

    # Assert the result
    assert user_data['name'] == 'Jane Doe'
```

## 4. Recommended Workflow

1.  **Write Initial Tests**: Begin by writing tests for the primary functionality, focusing on the public methods and their expected outcomes (the "happy path").
2.  **Manage Test State with `setUp` and `tearDown`**: Use the `setUp` method in your test class to create a clean instance before each test method runs. This ensures that your tests are independent and don't influence each other. The `tearDown` method can be used to clean up any resources after each test.
3.  **Identify Gaps**: Review the code to find uncovered lines, branches, and method interactions.
4.  **Write Targeted Tests**: For each missed scenario, determine the specific inputs, method call sequences, or mock behaviors needed to execute that code path. Write a new test case targeting that exact situation.
5.  **Refactor for Clarity**: As you add tests, keep them clean and readable. Use descriptive names (e.g., `test_user_deactivation_sets_is_active_to_false`) and consider using parameterized tests to avoid duplicating test logic when testing similar inputs.

## 5. Advanced Techniques and Best Practices

1.  **Parameterize Tests**: To test a method with multiple inputs using the same logic, use a loop with `subTest` or a parameterized testing library like `pytest`. This keeps your test suite DRY (Don't Repeat Yourself).
2.  **Verify State and Side Effects**: A test should not only check a method's return value but also verify any intended side effects, such as changes to the object's attributes or calls to a mocked dependency.
3.  **Test for Exceptions**: Use `assertRaises` (as a context manager or method) to confirm that a method correctly raises an exception with invalid input.
4.  **Keep Tests Independent**: Each test case should be able to run on its own and in any order. Avoid creating dependencies between tests where one test relies on the state left by a previous one.
5.  **Use Descriptive Naming**: Test methods should have long, descriptive names that clearly state what they are testing. This makes test results easier to understand and serves as a form of documentation.
</best_practices>

<source_code>
{source_code}
</source_code>

In [ ]:
%%writefile prompts/create_spec.md

<role>
You are a world-class software architect specialized in designing comprehensive specifications intended for junior developers.
</role>

<task>
Review the source code of the Python module provided in the `source_code` tag below and write a comprehensive Software Requirements Specification for this module by strictly following the process defined in the "process" tag below.
</task>

<constraints>
1. The specification must be written in plain English language, without any Python code snippets.
2. You can include diagrams in Mermaid format.
3. You must follow the Specification Design Process provided in the `process` tag below.
</constraints>

<process>
## Guiding Principles

1.  **Strict Contract**: The resulting SRS is a non-negotiable contract. Deviations in the final implementation, even for technical improvement, are considered defects.
2.  **Independent Resolution**: You MUST resolve all ambiguities autonomously based on currently available information (code, comments, tests). Do not rely on external clarification. Make reasonable engineering assumptions based on the simplest non-crashing behavior or standard patterns to ensure deterministic outcomes.
3.  **Absolute Completeness**: No placeholders (e.g., "etc.", "such as", "e.g."). Every list, enum, dictionary key set, attribute, method, and constant MUST be exhaustively enumerated.
4.  **Precision and Clarity**: Avoid vague terms. Define any domain-specific terms used. Specifications MUST be unambiguous and specific.

## Phase 1: Scope Definition and Strategic Planning

Establish the boundaries of the documentation effort, ensuring no part of the exposed surface area is omitted.

1.  **Define Core Objective**: Produce a complete SRS for all components within the target Python module.
2.  **Identify the Complete Module Namespace**: The scope is strictly defined by the *entire* module namespace. This includes ALL publicly accessible components, protected components, and any private components accessed by the test suite:
    *   Classes and their full inheritance hierarchies.
    *   Module-level functions and helper utilities.
    *   Exceptions, constants, type variables, and protocol definitions.
    *   Any internal component (even those prefixed with `_`) if it is accessed by the existing test suite, treating such access as defining a "hidden" public contract.
    *   Required module-level imports, including from `__future__`.
3.  **Conditional Features**: Identify features or classes available only under certain conditions (e.g., OS, Python version, presence of other modules). The SRS MUST specify exactly how their absence is reflected in the module namespace. These components SHALL NOT be defined in the namespace if the condition is not met.
    *   *Example:* "`KqueueSelector` class SHALL be present in the module namespace if and only if `select.kqueue` is available."

## Phase 2: Static Analysis and Architectural Deconstruction

Examine the source code without execution to map its structure, contracts, and dependencies with absolute precision.

1.  **Namespace and Hierarchy Cataloging**:
    *   Create a definitive list of every name exported by the module.
    *   Formalize all class hierarchies, explicitly documenting required naming conventions and exact inheritance trees.
    *   List all required module-level constants with their types and expected values.
        *   *Example:* `DEFAULT_TIMEOUT: final[int] = 10`
    *   List all required imports, including from `__future__`. Verify the existence and exact names of any functions or attributes assumed to be imported from other modules.
        *   *Example:* "SHALL import `annotations` from `__future__`."
    *   *Constraint Check*: Ensure NO "etc." or "e.g." is used. All items MUST be listed.

2.  **Comprehensive API Extraction**:
    *   **Classes**:
        *   List all attributes (public, protected, private): Specify name, type, access level (public, protected, private), and if it's a property or a plain instance/class attribute.
            *   *Example:* `_is_open: bool` - Instance Attribute, Private, Read/Write.
            *   *Example:* `@property max_size(self) -> int` - Property, Public, Read-Only.
        *   List all methods (public, protected, private): Provide the full signature, including parameter types, return types, and decorators (`@staticmethod`, `@classmethod`). Pay close attention to dunder methods, ensuring correct parameters (e.g., `self`).
            *   *Example:* `def _process_data(self, data: bytes, config: Dict[str, Any]) -> bool:`
            *   *Example:* `def __len__(self) -> int:`
    *   **Module-Level Callables (Functions)**: Extract the *exact* signature for every callable, including type hints for all parameters and return values. Explicit documentation for `*args` and `**kwargs` handling is mandatory.
    *   **Data Structures**: Deconstruct all Data Transfer Objects (DTOs), enums, and configuration dictionaries. Every allowed key or member MUST be enumerated. Use type hints to define the structure of complex types.
        *   *Example:* Config dict structure: `{"timeout": int, "retries": int, "protocol": Literal["tcp", "udp"]}`
        *   Specify mutability where important (e.g., `set` vs. `frozenset`, `List` vs. `Tuple`).

## Phase 3: Behavioral Analysis and Protocol Reverse-Engineering

Understand dynamic behavior, state transitions, and implicit Python protocols through rigorous examination of code and tests.

1.  **Test Suite Deconstruction as Source of Truth**:
    *   Analyze every test case to extract hard input-output scenarios and edge case handling.
    *   Treat mocked interactions in tests as rigid requirements for external dependencies.
    *   Identify "hidden" public contracts: If a test accesses an attribute named `_internal_cache`, that attribute MUST be documented as part of the specification as if it were public for testing purposes.

2.  **Implicit Protocol Identification**:
    *   Identify all implemented "dunder" methods (e.g., `__add__`, `__iter__`, `__call__`, `__getstate__`, `__str__`, `__repr__`).
    *   Document these not just as methods, but as formal adherence to Python protocols (e.g., "Implements the `Iterable` protocol via `__iter__`").

3.  **Logic, Side Effect, and Algorithm Mapping**:
    *   Use clear, sequential steps (e.g., numbered lists) and imperative language ("SHALL", "MUST") to describe method/function behavior.
    *   Specify the order of operations, especially when interactions or side effects are involved.
        *   *Example:* "1. SHALL validate input `config`. 2. SHALL acquire lock. 3. SHALL call `_internal_method()` *before* releasing lock."
    *   Enumerate all known edge cases and specify exactly how they MUST be handled.
        *   *Example:* "If input is None, SHALL raise `TypeError`. If input list is empty, SHALL return an empty `Dict`."
    *   Provide concrete examples of inputs and expected outputs, especially for parsing, formatting, or complex transformations.
    *   Document ALL observable side effects exactly (e.g., specific log message templates, exact file system paths utilized, mutations to input objects if applicable).
    *   **String Formats**: For methods returning strings (`__str__`, `__repr__`, custom formatting), provide the exact format string or a template.
        *   *Example:* `__repr__` SHALL return `f"MyClass(id={self.id!r})"`
    *   **Regular Expressions**: Include any required regular expressions in their entirety, with flags. Explain capture groups.
        *   *Example:* `VERSION_REGEX = re.compile(r"V(?P<major>\d+)", re.IGNORECASE)`
    *   **Algorithms/Formulas**: If a specific algorithm is required, provide step-by-step pseudo-code. Explicitly state any mathematical formulas. Detail data transformations like sorting, stripping, encoding, or type conversions.
        *   *Example:* "Output from command SHALL be decoded as UTF-8, stripped of leading/trailing whitespace, and split by newline characters."

## Phase 4: Specification Synthesis

Consolidate findings into a formal SRS document. This phase requires making definitive decisions to resolve all ambiguities without external input.

1.  **Ambiguity Resolution and Assumption Locking**:
    *   Explicitly document every ambiguity encountered during analysis.
    *   Resolve each ambiguity by making and documenting a reasonable engineering assumption. Base assumptions on promoting robustness, simplicity, deterministic behavior, or adherence to common Python idioms.
    *   *Example Resolution*: "The code handles `None` for `user_id` inconsistently. SRS mandates `user_id` MUST NOT be None and SHALL raise `ValueError` if None is provided."
    *   Avoid vague terms. Define any domain-specific terms used.

2.  **SRS Structural Requirements**:
    The SRS MUST include standard sections (Introduction, Overview, NFRs) AND specifically mandated sections:
    *   **Module Namespace Reference**: A dedicated section defining every exposed name (classes, functions, exceptions, constants), including conditional components as defined in Phase 1.
    *   **Assumption Log**: A record of all independent decisions made to resolve established ambiguities, with justifications.

3.  **Detailed Component Specification Rules**:
    Every component MUST be documented using the following strict formats:

    *   **For Classes**:
        *   **Inheritance**: The exact parent class(es).
        *   **Attributes**: A complete table of ALL public, protected, and private attributes (Name, Type, Access Level, Description, Property/Instance/Class).
            *   *Example:* `is_initialized: bool` | Instance Attribute | Private | Tracks initialization state.
        *   **Methods**: Detailed specification for each method including dunder methods (see Callables below).
        *   **Protocol Support**: Explicit list of supported Python protocols.

    *   **For Callables (Methods/Functions)**:
        *   **Full Signature**: MUST include all parameters with type hints and default values. Decorators MUST be listed.
            *   *Example:* `@classmethod def from_config(cls, config: Dict[str, Any]) -> 'MyClass':`
        *   **`*args`/`**kwargs` definition**: Explicit explanation of how variadic arguments are processed, including expected types and structure if applicable.
        *   **Preconditions**: What MUST be true before execution.
            *   *Example:* "Parameter `config` MUST not be None."
        *   **Behavior**: Detailed step-by-step description using imperative language, including order of operations, side effects, algorithms, and data transformations as per Phase 3.3.
        *   **Postconditions**: What is guaranteed to be true after successful execution.
        *   **Return Value**: Exact type and structure. For complex types, provide a breakdown.
            *   *Example:* Returns `Dict[str, List[Tuple[int, str]]]` where each tuple is `(index, value)`.
        *   **Error Handling**: Exact exception types and their *literal* error message strings (use f-string like placeholders for dynamic parts) for every error condition.
            *   *Example:* SHALL raise `ValueError` with message `f"Invalid ID: {id}"` if ID is not positive.
            *   *Example:* SHALL raise `TypeError` with message "Configuration must be a dict."

    *   **For Collections/Enums/Constants**:
        *   **Complete Enumeration**: ABSOLUTELY NO use of "e.g." or "etc.". Every member, key, or value MUST be listed.
        *   **Type and Value**: For constants, specify type and exact value.

## Phase 5: Verification and Validation

Ensure the SRS is accurate, complete, and actionable based *solely* on the generated document.

1.  **Canonical Test Suite Definition**:
    *   Translate identified behavioral scenarios, edge cases, and input/output examples from Phase 3 into a required acceptance test suite definition within the SRS.
    *   Ensure these tests cover the "hidden contracts" (e.g., `_private` attributes accessed by legacy tests) as specified.

2.  **Completeness Review**:
    *   Verify that NO placeholders ("etc.", "e.g.") exist anywhere in the document.
    *   Validate that every name in the module namespace is documented.
    *   Confirm that every assumption made in Phase 4 is clearly documented in the Assumption Log and reflected as a firm requirement in the component specifications.
    *   Ensure all types, string formats, error messages, and behaviors are specified with an absolute level of detail.
</process>

<source_code>
{source_code}
</source_code>

In [ ]:
%%writefile prompts/create_dd.md

<role>
You are a world-class software architect specialized in Python.
</role>

<task>
You will be given a Technical Specification located in the `technical_specification` tag below ("the Spec"). This Spec provides detailed requirements for designing one Python module.
Your goal is to write a Technical Design Document (TDD) that suggests an optimal design for this Python module.
</task>

<instructions>
- The design must strictly adhere to all requirements defined in the Spec, without adding any out-of-scope features.
- The target runtime environment must be Python 3.11 on Linux.
- The TDD must start with a brief overview of multiple viable design alternatives and compare their strengths and weaknesses.
- The TDD must recommend a single design option and justify the choice with a clear trade-off analysis relative to the requirements.
- The TDD must document the recommended design option in full details describing the system's structure and functionality using natural language, Mermaid diagrams, and short code snippets.
- A Mermaid class diagram is recommended if the module design involves multiple related classes.
- A Mermaid sequence diagram is recommended for non-trivial call chains or complex interactions.
- The TDD must include a dedicated section outlining relevant security implications and proposed mitigations.
</instructions>

<response_format>
- Write the TDD in Markdown format.
</response_format>

<technical_specification>
{spec}
</technical_specification>

In [ ]:
%%writefile prompts/write_new_code.md

<role>
You are a world-class software engineer specialized in implementation of Python modules.
</role>

<task>
You are given two documents:
1. Software Requirements Specification located in the `software_requirements_specification` tag below.
2. Technical Design Document located in the `technical_design_document` tag below.
Your goal is to develop a new Python module based strictly on these two documents.
</task>

<instructions>
- You can leverage all Standard Python Libraries available in Python 3.11, but you are not allowed to import any PyPi packages.
- Before submitting your response perform a thorough code review and focus on the following 3 critical aspects:
  1. Your code implementation must implement all requirements found in the specification and design doc, without any gaps.
  2. Your code must be bug free.
  3. Your code must not introduce any extra functionality that is not specifically requested in the design doc.
</instructions>

<response_format>
Your response must be a single Markdown code block with the full code of the Python module that you implemented. Do not include any unit tests or code samples that show how to use this module.
</response_format>

<software_requirements_specification>
{spec}
</software_requirements_specification>

<technical_design_document>
{design_doc}
</technical_design_document>

### Step 1: Select and load Target Module

We will use `twisted.python.zipstream` and download the raw file directly from GitHub.

https://github.com/twisted/twisted/blob/master/src/twisted/python/zipstream.py

*   **Source**: `original_code/zipstream.py`
*   **Target**: `output/zipstream.py`

In [ ]:
MODULE_URL = "https://raw.githubusercontent.com/twisted/twisted/master/src/twisted/python/zipstream.py"
TARGET_MODULE = "zipstream"
CODE_DIR_PATH = "original_code"
CODE_PATH = f"{CODE_DIR_PATH}/{TARGET_MODULE}.py"

!wget -q -O {CODE_PATH} {MODULE_URL}

### Step 2: Generate and Verify Unit Tests

If the legacy code lacks tests, we must generate them to establish a baseline of correctness.

#### 2.1 Generate Tests

We read the legacy code and use a specific prompt to generate high-coverage `unittest` cases.

In [ ]:
# Load code
with open(CODE_PATH) as f:
    source_code = f.read()
print(f"Loaded {TARGET_MODULE} ({len(source_code)} bytes)")

TEST_PATH = f"output/tests/test_{TARGET_MODULE}.py"

generated_tests = generate_and_save(
    template_path="prompts/create_tests.md",
    output_path=TEST_PATH,
    replacements={
        "source_code": source_code,
        "module_name": TARGET_MODULE,
        "code_dir_path": CODE_DIR_PATH,
    },
)
print(f"Tests saved to {TEST_PATH}")

#### 2.2 Verify Baseline (Run Tests)

We must ensure these newly generated tests actually pass on the *legacy* code before moving on. We use `subprocess` to isolate the test execution.

In [ ]:
import os
import subprocess
import sys

from IPython.display import Markdown

def run_tests(test_dir: str, code_dir: str, test_file: str) -> None:
    """Runs tests in a separate process and displays a rich Markdown table report."""
    env = os.environ.copy()
    # Ensure the module under test can be imported
    env["PYTHONPATH"] = f"{code_dir}" + os.pathsep + env.get("PYTHONPATH", "")

    # Use -v to get detailed output for each test
    cmd = [sys.executable, "-m", "unittest", "-v", os.path.join(test_dir, test_file)]

    display(Markdown(f"### Running `{test_file}` against `{code_dir}`..."))
    result = subprocess.run(cmd, check=False, env=env, capture_output=True, text=True)

    table_lines = []
    table_lines.append("| Test Name | Description | Status |")
    table_lines.append("| :--- | :--- | :---: |")

    lines = result.stderr.splitlines()
    pending_test_name = None
    passed = 0
    failed = 0
    skipped = 0

    for line in lines:
        line = line.strip()
        if not line:
            continue

        if " ... " in line:
            parts = line.split(" ... ")
            if len(parts) >= 2:
                desc_part = parts[0].strip()
                status_part = parts[1].strip()

                if pending_test_name:
                    test_name = pending_test_name
                    description = desc_part
                    pending_test_name = None
                else:
                    # Fallback parse if no header line preceded
                    test_parts = desc_part.split(" ")
                    test_name = test_parts[0]
                    description = (
                        " ".join(test_parts[1:]).strip() if len(test_parts) > 1 else ""
                    )
                    # Handle trailing parens if present
                    if "(" in description:
                        description = description.split("(")[0].strip()

                if status_part == "ok":
                    status_emoji = "✅ PASS"
                    passed += 1
                elif status_part.startswith("FAIL"):
                    status_emoji = "❌ FAIL"
                    failed += 1
                elif status_part.startswith("ERROR"):
                    status_emoji = "🛑 ERROR"
                    failed += 1
                elif status_part.startswith("skipped"):
                    status_emoji = "⚠️ SKIP"
                    skipped += 1
                else:
                    status_emoji = f"❓ {status_part.split()[0].upper()}"

                table_lines.append(
                    f"| `{test_name}` | {description} | {status_emoji} |"
                )

        elif line.startswith("test_") and "(" in line and ")" in line:
            pending_test_name = line.split(" ")[0]
        elif not line.startswith("test_") and line.strip() != "":
            pending_test_name = None

    display(Markdown("\n".join(table_lines)))
    display(
        Markdown(f"\n**Summary:** {passed} passed, {failed} failed, {skipped} skipped.")
    )

    if result.returncode == 0 and failed == 0:
        display(Markdown("\n### ✅ **OVERALL SUCCESS: All tests passed.**"))
        return True
    display(Markdown("\n### ❌ **OVERALL FAILURE: Some tests failed.**"))
    if failed > 0 or result.returncode != 0:
        print("\nDetailed error output:")
        print(result.stderr)
    return False


def verify_module(test_dir: str, code_dir: str, module_name: str) -> bool:
    """Wrapper to check file existence and run tests."""
    code_path = os.path.join(code_dir, f"{module_name}.py")
    test_file = f"test_{module_name}.py"
    if os.path.exists(code_path) and os.path.exists(os.path.join(test_dir, test_file)):
        success = run_tests(test_dir, code_dir, test_file)
        if success:
            print(f"\nVerified! Valid module found at: {code_path}")
        return success
    print("Cannot run verification due to missing files.")
    return False

In [ ]:
# Run tests on original code
verify_module("output/tests", CODE_DIR_PATH, TARGET_MODULE)

### Step 3: Generate Specification

Create a detailed Markdown specification derived strictly from the legacy code's behavior.

In [ ]:
SPEC_PATH = f"output/specs/{TARGET_MODULE}.md"
spec_content = generate_and_save(
    template_path="prompts/create_spec.md",
    output_path=SPEC_PATH,
    replacements={"source_code": source_code},
)
display(Markdown(spec_content))

### Step 4: Generate Design Document

Using *only* the specification from Step 3, generate a design document. This ensures the new design is based on desired behavior, not just copying the old implementation details.

In [ ]:
DD_PATH = f"output/design_docs/{TARGET_MODULE}.md"
dd_content = generate_and_save(
    template_path="prompts/create_dd.md",
    output_path=DD_PATH,
    replacements={"spec": spec_content},
)
render_markdown_with_mermaid(dd_content)

### Step 5: Generate New Code

Generate the refactored code using the Specification and Design Doc. The legacy code is *not* used in this input, preventing contamination from old anti-patterns.

In [ ]:
NEW_CODE_PATH = f"output/code/{TARGET_MODULE}.py"
new_code = generate_and_save(
    template_path="prompts/write_new_code.md",
    output_path=NEW_CODE_PATH,
    replacements={"spec": spec_content, "design_doc": dd_content},
)

### Step 6: Final Verification

Run the tests generated in Step 2 against the new code generated in Step 5.

In [ ]:
verify_module("output/tests", "output/code", TARGET_MODULE)

## 7. Introduce new Requirements for the code

Delete and rewrite the code, show that the added requirements are successfully incorporated.

We will update the specification with new requirements, then re-generate the Design Doc and code based upon that specification.

In [ ]:
# Add new requirements to specification
new_requirements = """## Coding style requirements
1. All input parameters and return values must have type hints.
2. All methods and classes must have Google-style Docstrings.
"""

with open(SPEC_PATH, "a+") as f:
    f.write(new_requirements)
    f.seek(0)
    spec_content = f.read()

# Regenerate Design Doc & Code
dd_content = generate_and_save("prompts/create_dd.md", DD_PATH, {"spec": spec_content})
new_code = generate_and_save(
    "prompts/write_new_code.md",
    NEW_CODE_PATH,
    {"spec": spec_content, "design_doc": dd_content},
)

In [ ]:
verify_module("output/tests", "output/code", TARGET_MODULE)